
<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con la demanda eléctrica.** Las provincias con un PIB más alto o en crecimiento experimentan una mayor demanda de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Extracción

En el laboratorio de hoy tendrás que extraer la información necesaria para obtener tu objetivo de las siguientes fuentes de datos (deberás usar API's y herramientas de *web scrapping*):

- **Datos de la API de Red Eléctrica Española (REE):** Deberás extraer datos mensuales a nivel provincial de los siguientes aspectos:

  - **Demanda Eléctrica:** Nos proporciona los datos de demanda eléctrica a nivel provincial, agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/demanda/evolucion", añadiendo los parámetros que sean necesarios. 

  - **Generación Eléctrica:** Nos proporciona los datos de generación eléctrica a nivel provincial, diferenciando entre fuentes de energía (eólica, solar, hidroeléctrica, etc.), agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/generacion/estructura-renovables", añadiendo los parámetros que sean necesarios.

  La documentación de la API la encontrarás en [este link](https://www.ree.es/es/apidatos). Recuerda leer en detenimiento la documentación. 

- **Datos del Instituto Nacional de Estadística (INE):** Además de los datos de la REE, debes extraer y utilizar datos socioeconómicos de las siguientes páginas del INE:

- **Datos Demográficos:** Extraer los datos de población por provincias, diferenciando por grupos de edad, sexo, y extrajeros. Estos datos serán utilizados para analizar cómo la población afecta a la demanda eléctrica en cada provincia.

  - **Página web:** [INE - Población por provincias](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990)

  - "Principales series 1998-2022" --> "Por provincia" --> " Población por provincias, edad (3 grupos de edad), españoles/Extranjeros, Sexo y año"

- **Datos Económicos:**

  - **Página web:** [INE - PIB por provincias](https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426&capsel=10429). 

  - **Pasos para la extracción**:" Resultados provinciales. Serie contable 2016-2021" --> "P.I.B. a precios de mercado y valor añadido bruto a precios básicos por ramas de actividad: Precios corrientes por provincias y periodo."



NOTA1: Tienes que sacar muchos datos, pero recuerda que hemos aprendido herramientas de asincronia que te pueden ayudar en este paso de la ETL. 

NOTA2: Todos estos datos los debes sacar para los años 2019-2020-2021

In [5]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from time import sleep
# Selenium para establecer la configuración del driver
# -----------------------------------------------------------------------
from selenium import webdriver
import os

import pandas as pd
import numpy as np
import requests
import json

import warnings
warnings.filterwarnings('ignore')

# Llamadas a la API

In [6]:
lista_anios = [2019,2020,2021]

cod_comunidades = {'Ceuta': 8744,
                        'Melilla': 8745,
                        'Andalucía': 4,
                        'Aragón': 5,
                        'Cantabria': 6,
                        'Castilla - La Mancha': 7,
                        'Castilla y León': 8,
                        'Cataluña': 9,
                        'País Vasco': 10,
                        'Principado de Asturias': 11,
                        'Comunidad de Madrid': 13,
                        'Comunidad Foral de Navarra': 14,
                        'Comunitat Valenciana': 15,
                        'Extremadura': 16,
                        'Galicia': 17,
                        'Illes Balears': 8743,
                        'Canarias': 8742,
                        'Región de Murcia': 21,
                        'La Rioja': 20}

In [7]:
for anio in lista_anios:
    for comunidad, id in cod_comunidades.items():
        url = f"https://apidatos.ree.es/es/datos/demanda/evolucion?start_date={anio}-01-01T00:00&end_date={anio}-12-31T23:59&time_trunc=month&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={id}"

        cabecera = {"Accept": "application/json",
                    "Content-Type": "application/json",
                    "Host": "apidatos.ree.es"}

        response = requests.get(url, headers=cabecera)
        #print(f"La respuesta a la API es: {response.status_code}") 

        res = response.json() 

        carpeta = f"C:\\Users\\Elena\\OneDrive\\Desktop\\Hackio\\Modulos\\Modulo5\\2-Labs\\datos\\evolucion\\{anio}"
        os.makedirs(carpeta, exist_ok=True)

        nombre = "" + comunidad + ".json"
        ruta_archivo = os.path.join(carpeta, nombre)

        with open(ruta_archivo, "w") as archivo_json:
            json.dump(res, archivo_json, indent=4)
        

In [8]:
for anio in lista_anios:
    for comunidad, id in cod_comunidades.items():
      
        url = f"https://apidatos.ree.es/es/datos/generacion/estructura-renovables?start_date={anio}-01-01T00:00&end_date={anio}-12-31T23:59&time_trunc=month&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={id}"

        cabecera = {"Accept": "application/json",
                "Content-Type": "application/json",
                "Host": "apidatos.ree.es"}

        response = requests.get(url, headers=cabecera)
        
        if response.status_code == 200:

                res = response.json() 

                carpeta = f"C:\\Users\\Elena\\OneDrive\\Desktop\\Hackio\\Modulos\\Modulo5\\2-Labs\\datos\\generacion\\{anio}"
                os.makedirs(carpeta, exist_ok=True)

                nombre = "" + comunidad + ".json"
                ruta_archivo = os.path.join(carpeta, nombre)

                with open(ruta_archivo, "w") as archivo_json:
                        json.dump(res, archivo_json, indent=4)

        else:
                print(f"La respuesta para la comunidad autónoma {comunidad} en el año {anio} es {response.status_code}")
        

La respuesta para la comunidad autónoma Ceuta en el año 2019 es 502
La respuesta para la comunidad autónoma Ceuta en el año 2020 es 502
La respuesta para la comunidad autónoma Ceuta en el año 2021 es 502


# Scrapeo INE

### Opición 1

In [12]:
url1 = "https://www.ine.es/jaxi/Tabla.htm?path=/t20/e245/p08/l0/&file=03001.px&L=0"
url2 = "https://www.ine.es/jaxi/Tabla.htm?tpx=67284&L=0"
lista_urls = [url1, url2]

for url in lista_urls:
    # lo primero que vamos a hacer es configurar nuestras preferencias del navegador para el driver
    chrome_options = webdriver.ChromeOptions()

    # establacemos las preferencias que queremos
    prefs = {
        "download.default_directory": "C:\\Users\\Elena\\OneDrive\\Desktop\\Hackio\\Modulos\\Modulo5\\2-Labs\\datos\\DatosDescargados",  
        "download.prompt_for_download": False,   # desactiva el diálogo que Chrome normalmente muestra para pedir confirmación del usuario antes de descargar un archivo
        "directory_upgrade": True,    # hace que Chrome actualice el directorio de descarga predeterminado a la nueva ubicación especificada por download.default_directory si esta ha cambiado.
    }

    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(options=chrome_options)

    driver.get(url)
    sleep(1)

    driver.find_element("css selector", "#btnDescarga > i").click()
    sleep(3)

    iframe_descargas = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("css selector", '#thickBoxINEfrm')))
    driver.switch_to.frame(iframe_descargas)
    sleep(2)
    driver.find_element("css selector", "body > ul > li:nth-child(4) > a").click()
    sleep(2)


In [13]:
ruta = "/Users/Elena/OneDrive/Desktop/Hackio/Modulos/Modulo5/2-Labs/datos/DatosDescargados"
for archivo in os.listdir(ruta):
    if archivo == '03001.csv':
        os.rename(f"{ruta}/{archivo}", f"{ruta}/ine_poblacion.csv")
    if archivo == '67284.csv':
        os.rename(f"{ruta}/{archivo}", f"{ruta}/ine_pib.csv")

### Opción 2

In [49]:
url1 = "https://www.ine.es/jaxi/Tabla.htm?path=/t20/e245/p08/l0/&file=03001.px&L=0"
url2 = "https://www.ine.es/jaxi/Tabla.htm?tpx=67284&L=0"
lista_urls = [url1, url2]

for i in range(len(lista_urls)):
    
    # lo primero que vamos a hacer es configurar nuestras preferencias del navegador para el driver
    chrome_options = webdriver.ChromeOptions()

    # establacemos las preferencias que queremos
    prefs = {
        "download.default_directory": "C:\\Users\\Elena\\OneDrive\\Desktop\\Hackio\\Modulos\\Modulo5\\2-Labs\\datos\\DatosDescargados",  
        "download.prompt_for_download": False,   # desactiva el diálogo que Chrome normalmente muestra para pedir confirmación del usuario antes de descargar un archivo
        "directory_upgrade": True,    # hace que Chrome actualice el directorio de descarga predeterminado a la nueva ubicación especificada por download.default_directory si esta ha cambiado.
    }

    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(options=chrome_options)

    driver.get(lista_urls[i])
    sleep(1)
    driver.maximize_window()
    sleep(1)

    # Para todos los campos menos año seleccionamos todas las opciones
    driver.find_element("css selector", "#tg0 > div > fieldset > div.capaSelecTodosNinguno > button.opcionesvarDer > i").click()
    sleep(1)
    driver.find_element("css selector", "#tg1 > div > fieldset > div.capaSelecTodosNinguno > button.opcionesvarDer > i").click()
    sleep(1)
    if i == 0:
        driver.find_element("css selector", "#tg2 > div > fieldset > div.capaSelecTodosNinguno > button.opcionesvarDer > i").click()
        sleep(1)
        driver.find_element("css selector", "#tg3 > div > fieldset > div.capaSelecTodosNinguno > button.opcionesvarDer > i").click()
        sleep(1)


    # Borramos la seleccion de años
    driver.find_element("css selector", "#caja_periodo > div > fieldset > div.capaSelecTodosNinguno > button:nth-child(3) > i").click()
    sleep(1)
    
    # Localizar las opciones (puedes usar cualquier método de selección que necesites)
    if i == 0:
        aux1 = driver.find_element("css selector", "#periodo > option:nth-child(4)")
    else:
        aux1 = driver.find_element("css selector", "#periodo > option:nth-child(1)")

    aux2 = driver.find_element("css selector", "#periodo > option:nth-child(3)")
    aux3 = driver.find_element("css selector", "#periodo > option:nth-child(2)")
    lista_anios= [aux1, aux2, aux3]
    sleep(1)
    
    acciones = ActionChains(driver)

    # Pulsar Control, hacer clic en cada opción, y soltar Control al final
    acciones.key_down(Keys.CONTROL)  # Mantener la tecla Control presionada

    for opcion in lista_anios:
        acciones.click(opcion)  # Clic en cada opción

    acciones.key_up(Keys.CONTROL)  # Soltar la tecla Control
    acciones.perform() 
    
    # Pulsamos para hacer la seleccion
    driver.execute_script("window.scrollTo(0, 1000);")
    driver.find_element("css selector", "#botonConsulSele").click()

    #Pulsamos en descargar
    sleep(2)
    driver.find_element("css selector", "#btnDescargaForm > i").click()
    sleep(2)
    iframe_descargas = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("css selector", '#thickBoxINEfrm')))
    driver.switch_to.frame(iframe_descargas)
    sleep(2)
    driver.find_element("css selector", "#export > ul > li:nth-child(4) > label").click()
    

  

0
1
